In [ ]:
import numpy as np
import nltk
import pandas as pd
import collections
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#import data
df=pd.read_csv("drive/My Drive/NLP summer Internship/complaints.csv")
df

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
1,2019-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,15206,NaN,Consent not provided,Web,2019-09-20,Closed with non-monetary relief,Yes,NaN,3379500
2,2019-10-25,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I would like to request the suppression of the...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,937XX,NaN,Consent provided,Web,2019-10-25,Closed with explanation,Yes,NaN,3417821
3,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,"Over the past 2 weeks, I have been receiving e...",NaN,"Diversified Consultants, Inc.",NC,275XX,NaN,Consent provided,Web,2019-11-08,Closed with explanation,Yes,NaN,3433198
4,2019-02-08,Vehicle loan or lease,Lease,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,NaN,HYUNDAI CAPITAL AMERICA,FL,33161,NaN,Consent not provided,Web,2019-02-08,Closed with non-monetary relief,Yes,NaN,3146310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660581,2017-03-06,Credit reporting,NaN,Incorrect information on credit report,Reinserted previously deleted info,"I have been a victim if inquiry issue,",Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,945XX,NaN,Consent provided,Web,2017-03-09,Closed with explanation,Yes,Yes,2373702
1660582,2017-03-27,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt is not mine,I have disputed this debt directly with Divers...,NaN,"Diversified Consultants, Inc.",VA,232XX,NaN,Consent provided,Web,2017-03-27,Closed with explanation,Yes,No,2405925
1660583,2015-03-26,Mortgage,FHA mortgage,"Loan servicing, payments, escrow account",NaN,"On XXXX XXXX, 2015, I contacted XXXX XXXX, who...",NaN,Finance of America Mortgage LLC,CA,925XX,NaN,Consent provided,Web,2015-03-26,Closed with monetary relief,Yes,No,1303324
1660584,2015-12-12,Mortgage,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",NaN,I can not get from chase who services my mortg...,NaN,JPMORGAN CHASE & CO.,NY,110XX,NaN,Consent provided,Web,2015-12-12,Closed with explanation,Yes,No,1694674


In [ ]:
#EDA, inspect missing data
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Missing Percent'])
missing_data['Missing Percent'] = missing_data['Missing Percent'].apply(lambda x: x * 100)
missing_data.loc[missing_data['Missing Percent'] >=0]

,Total,Missing Percent
Tags,1436841,86.526142
Consumer complaint narrative,1103064,66.426189
Company public response,1008572,60.735909
Consumer disputed?,892109,53.722541
Consumer consent provided?,642482,38.690077
Sub-issue,574292,34.583695
Sub-product,235165,14.161567
ZIP code,159244,9.589627
State,28286,1.703375
Company response to consumer,2,0.000120


In [ ]:
df["Product"].unique()

array(['Debt collection',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Vehicle loan or lease', 'Checking or savings account',
       'Credit card or prepaid card',
       'Money transfer, virtual currency, or money service', 'Mortgage',
       'Student loan', 'Payday loan, title loan, or personal loan',
       'Credit card', 'Consumer Loan', 'Payday loan', 'Credit reporting',
       'Other financial service', 'Bank account or service',
       'Prepaid card', 'Money transfers', 'Virtual currency'],
      dtype=object)

In [ ]:
#Extract columns
data=df[["Product","Consumer complaint narrative","Complaint ID","Company"]]
data=data.rename(columns={"Consumer complaint narrative":"Complaint"})
data.head()

,Product,Complaint,Complaint ID,Company
0,Debt collection,transworld systems inc. \nis trying to collect...,3384392,TRANSWORLD SYSTEMS INC
1,"Credit reporting, credit repair services, or o...",NaN,3379500,Experian Information Solutions Inc.
2,"Credit reporting, credit repair services, or o...",I would like to request the suppression of the...,3417821,"TRANSUNION INTERMEDIATE HOLDINGS, INC."
3,Debt collection,"Over the past 2 weeks, I have been receiving e...",3433198,"Diversified Consultants, Inc."
4,Vehicle loan or lease,NaN,3146310,HYUNDAI CAPITAL AMERICA


In [ ]:
#Drop null values
data=data.dropna(axis=0)
data

,Product,Complaint,Complaint ID,Company
0,Debt collection,transworld systems inc. \nis trying to collect...,3384392,TRANSWORLD SYSTEMS INC
2,"Credit reporting, credit repair services, or o...",I would like to request the suppression of the...,3417821,"TRANSUNION INTERMEDIATE HOLDINGS, INC."
3,Debt collection,"Over the past 2 weeks, I have been receiving e...",3433198,"Diversified Consultants, Inc."
9,"Money transfer, virtual currency, or money ser...","I was sold access to an event digitally, of wh...",3366475,"Paypal Holdings, Inc"
10,Mortgage,This complaint dates back to last XX/XX/XXXX d...,3170293,NATIONSTAR MORTGAGE
...,...,...,...,...
1660581,Credit reporting,"I have been a victim if inquiry issue,",2373702,"TRANSUNION INTERMEDIATE HOLDINGS, INC."
1660582,Debt collection,I have disputed this debt directly with Divers...,2405925,"Diversified Consultants, Inc."
1660583,Mortgage,"On XXXX XXXX, 2015, I contacted XXXX XXXX, who...",1303324,Finance of America Mortgage LLC
1660584,Mortgage,I can not get from chase who services my mortg...,1694674,JPMORGAN CHASE & CO.


In [ ]:
#Inspect labels
data.groupby("Product")["Complaint ID"].nunique()

Product
Bank account or service                                                          14885
Checking or savings account                                                      22433
Consumer Loan                                                                     9473
Credit card                                                                      18838
Credit card or prepaid card                                                      38555
Credit reporting                                                                 31588
Credit reporting, credit repair services, or other personal consumer reports    181294
Debt collection                                                                 116679
Money transfer, virtual currency, or money service                                9179
Money transfers                                                                   1497
Mortgage                                                                         66256
Other financial service            

In [ ]:
#Rename product to label
data["Label"]=data["Product"]
data

,Product,Complaint,Complaint ID,Company,Label
0,Debt collection,transworld systems inc. \nis trying to collect...,3384392,TRANSWORLD SYSTEMS INC,Debt collection
2,"Credit reporting, credit repair services, or o...",I would like to request the suppression of the...,3417821,"TRANSUNION INTERMEDIATE HOLDINGS, INC.","Credit reporting, credit repair services, or o..."
3,Debt collection,"Over the past 2 weeks, I have been receiving e...",3433198,"Diversified Consultants, Inc.",Debt collection
9,"Money transfer, virtual currency, or money ser...","I was sold access to an event digitally, of wh...",3366475,"Paypal Holdings, Inc","Money transfer, virtual currency, or money ser..."
10,Mortgage,This complaint dates back to last XX/XX/XXXX d...,3170293,NATIONSTAR MORTGAGE,Mortgage
...,...,...,...,...,...
1660581,Credit reporting,"I have been a victim if inquiry issue,",2373702,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Credit reporting
1660582,Debt collection,I have disputed this debt directly with Divers...,2405925,"Diversified Consultants, Inc.",Debt collection
1660583,Mortgage,"On XXXX XXXX, 2015, I contacted XXXX XXXX, who...",1303324,Finance of America Mortgage LLC,Mortgage
1660584,Mortgage,I can not get from chase who services my mortg...,1694674,JPMORGAN CHASE & CO.,Mortgage


In [ ]:
#Combine similar labels
data_combined=data.copy()
data_combined["Label"]=data_combined["Label"].replace("Credit card","Credit card or prepaid card")
data_combined["Label"]=data_combined["Label"].replace("Credit reporting","Credit reporting, credit repair services, or other personal consumer reports")
data_combined["Label"]=data_combined["Label"].replace("Money transfers","Money transfer, virtual currency, or money service")
data_combined["Label"]=data_combined["Label"].replace("Payday loan","Payday loan, title loan, or personal loan")
data_combined["Label"]=data_combined["Label"].replace("Prepaid card","Credit card or prepaid card")
data_combined["Label"]=data_combined["Label"].replace("Virtual currency","Money transfer, virtual currency, or money service")
data_combined.groupby("Label")["Complaint ID"].nunique()

Label
Bank account or service                                                          14885
Checking or savings account                                                      22433
Consumer Loan                                                                     9473
Credit card or prepaid card                                                      58843
Credit reporting, credit repair services, or other personal consumer reports    212882
Debt collection                                                                 116679
Money transfer, virtual currency, or money service                               10692
Mortgage                                                                         66256
Other financial service                                                            292
Payday loan, title loan, or personal loan                                         9189
Student loan                                                                     26411
Vehicle loan or lease                

In [ ]:
#Inspect companies
data_combined[["Company","Product"]].groupby(['Company']).count().sort_values("Product",ascending=False)[:40]

,Product
Company,
"EQUIFAX, INC.",61377
Experian Information Solutions Inc.,56355
"TRANSUNION INTERMEDIATE HOLDINGS, INC.",55067
"CITIBANK, N.A.",18261
WELLS FARGO & COMPANY,17618
"BANK OF AMERICA, NATIONAL ASSOCIATION",17302
JPMORGAN CHASE & CO.,17185
CAPITAL ONE FINANCIAL CORPORATION,15016
"Navient Solutions, LLC.",14878


In [ ]:
#Data Clean, remove stop words, stemming tokens, lemmatizing tokens
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
porter=PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;#_]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text) 
    text = re.sub(r'[0-9]', ' ', text)
    text = re.sub(r'\b\d+\b',' ',text)
    text = re.sub(r"\w*[X]{2,20}\w*", ' ', text) 
    text = re.sub(r"\w*[x]{2,20}\w*", ' ', text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    return text

def tokenized(sentence):
    token_words=word_tokenize(sentence)
    return token_words
def remove_sw(tokenized_sentence):
    removed=[i for i in tokenized_sentence if i not in STOPWORDS] # delete stopwors from text
    return removed
def stemming(tokenized_sentence):
    stemmed=[porter.stem(i) for i in tokenized_sentence]
    stemmed_sentence=" ".join(stemmed)
    return stemmed_sentence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#Produce two columns, one column clean bad symbols, remove stop word before stemming
#The other column only clean bad symbols before stemming no stop word 
import time
start=time.time()
print("Creating stemmed dataset")
data_combined2=data_combined[["Label","Complaint","Complaint ID","Company"]].copy()
data_combined2['Complaint_full'] = data_combined2['Complaint'].apply(clean_text)
data_combined2['Complaint_full'] = data_combined2['Complaint_full'].apply(tokenized)
print("finish clean, now preprocessing")
data_combined2['Complaint_reduced'] = data_combined2['Complaint_full'].copy()
data_combined2['Complaint_reduced'] = data_combined2['Complaint_reduced'].apply(remove_sw)
print("stemming complaint after removing stop_word")
data_combined2['Complaint_reduced'] = data_combined2['Complaint_reduced'].apply(stemming)
print("stemming complaint without removing stop word")
data_combined2['Complaint_full'] = data_combined2['Complaint_full'].apply(stemming)
end=time.time()
print(end-start)
data_combined2

Creating stemmed dataset
finish clean, now preprocessing
stemming complaint after removing stop_word
stemming complaint without removing stop word
3002.557385444641


,Label,Complaint,Complaint ID,Company,Complaint_full,Complaint_reduced
0,Debt collection,transworld systems inc. \nis trying to collect...,3384392,TRANSWORLD SYSTEMS INC,transworld system inc is tri to collect debt t...,transworld system inc tri collect debt mine ow...
2,"Credit reporting, credit repair services, or o...",I would like to request the suppression of the...,3417821,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",i would like to request the suppress of the fo...,would like request suppress follow item credit...
3,Debt collection,"Over the past 2 weeks, I have been receiving e...",3433198,"Diversified Consultants, Inc.",over the past week have been receiv excess amo...,past week receiv excess amount telephon call c...
9,"Money transfer, virtual currency, or money ser...","I was sold access to an event digitally, of wh...",3366475,"Paypal Holdings, Inc",i wa sold access to an event digit of which ha...,sold access event digit screenshot detail tran...
10,Mortgage,This complaint dates back to last XX/XX/XXXX d...,3170293,NATIONSTAR MORTGAGE,thi complaint date back to last dure loan modi...,complaint date back last loan modif done accou...
...,...,...,...,...,...,...
1660581,"Credit reporting, credit repair services, or o...","I have been a victim if inquiry issue,",2373702,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",i have been victim if inquiri issu,victim inquiri issu
1660582,Debt collection,I have disputed this debt directly with Divers...,2405925,"Diversified Consultants, Inc.",i have disput thi debt directli with diversifi...,disput debt directli diversifi consult request...
1660583,Mortgage,"On XXXX XXXX, 2015, I contacted XXXX XXXX, who...",1303324,Finance of America Mortgage LLC,on contact who is branch manag with gateway fu...,contact branch manag gateway fund learn lower ...
1660584,Mortgage,I can not get from chase who services my mortg...,1694674,JPMORGAN CHASE & CO.,i can not get from chase who servic my mortgag...,get chase servic mortgag own origin loan doc s...


In [ ]:
#Output cleanned data without indexing
data_combined2.to_csv("drive/My Drive/NLP summer Internship/Cleanned_Complaints.csv",index=False)